See reproduction/sensor-cleaning.md for readme

In [1]:
library(tidyverse)
library(anomalize)
library(forecast)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



In [2]:
# Get list of CSV files in sensor_data directory
csv_files <- list.files(path = "data/sensor_data", pattern = "patch_[0-9]{2,3}.csv", full.names = TRUE)

# create output path
out_path <- "data/sensor_data_clean"
if (!dir.exists(out_path)) {
    dir.create(out_path, recursive = TRUE)
}

In [3]:
# Function to load data, remove outliers, and save cleaned data
clean_data <- function(file_path) {

    df <- read_csv(file_path)

    var_names <- grep("water_content", names(df), value = TRUE)

    df <- df %>% select(c(dateTime,all_of(var_names)))
    df <- na.omit(df)

    patch <- str_extract(file_path, "\\d+")

    print(paste("After removing NA: ", nrow(df)))

    df_list <- list()

    for (var in var_names) {
        # Perform time series decomposition and anomaly detection

        df %>% 
            time_decompose(.data[[var]], method = "stl", frequency = 60, trend = 180) %>% # 24 hour frequency (season), 3 hour trend with STL works
            anomalize(remainder, method = "iqr", alpha = 0.05) -> df_outliers  # Adjusted parameters

        var_names_2 <- grep("observed", names(df_outliers), value = TRUE)

        # Filter out the outliers
        df_no_outliers <- df_outliers %>%
            filter(anomaly == 'No') %>% 
            select(dateTime, all_of(var_names_2))  # Retain the dateTime and observed data columns

        print(paste("After outlier removal for", var, ":", nrow(df_no_outliers)))

        # Rename columns to make them unique (excluding 'dateTime')
        cols_to_rename <- setdiff(colnames(df_no_outliers), "dateTime")
        df_no_outliers <- df_no_outliers %>% rename_at(vars(cols_to_rename), ~paste0(., "_", var))

        # Store the result in the list
        df_list[[var]] <- df_no_outliers
    }

    # Combine all data frames in the list by dateTime
    df_combined <- Reduce(function(...) merge(..., by = "dateTime", all = TRUE), df_list)
    print(paste("Final data frame: ", nrow(df_combined)))

    df_combined <- df_combined %>%
        pivot_longer(
            cols = -dateTime,  # keep dateTime column as it is
            names_to = c("side", "depth"),  # create new columns
            names_pattern = "observed_water_content_(.*)_([0-9]+)_cm",  # split column names
            names_transform = list(
                depth = ~ readr::parse_double(.x)),
            values_to = "wc") %>% 
        mutate(patch = patch)

    # Define the output file path
    output_file_path <- str_replace(file_path, "sensor_data", "sensor_data_clean")

    # Save cleaned data back to CSV
      write_csv(df_combined, path = output_file_path)

}

# Apply the function to all CSV files in the directory
lapply(csv_files, clean_data)


Rows: 70527 Columns: 31
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl  (30): water_content_right_30_cm, soil_temperature_right_30_cm, permitti...
dttm  (1): dateTime

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "After removing NA:  60058"


Converting from tbl_df to tbl_time.
Auto-index message: index = dateTime

frequency = 60 minutes

trend = 180 minutes



[1] "After outlier removal:  55248"


Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(cols_to_rename)

  # Now:
  data %>% select(all_of(cols_to_rename))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."
Converting from tbl_df to tbl_time.
Auto-index message: index = dateTime

frequency = 60 minutes

trend = 180 minutes



...

Converting from tbl_df to tbl_time.
Auto-index message: index = dateTime

frequency = 60 minutes

trend = 180 minutes



[1] "After outlier removal:  57717"


Converting from tbl_df to tbl_time.
Auto-index message: index = dateTime

frequency = 60 minutes

trend = 180 minutes



[1] "After outlier removal:  55718"
[1] "Final data frame:  58262"


dateTime,side,depth,wc,patch
<dttm>,<chr>,<dbl>,<dbl>,<chr>
2020-09-22 10:40:00,right,30,4.7,102
2020-09-22 10:40:00,right,60,NA,102
2020-09-22 10:40:00,right,90,3.3,102
2020-09-22 10:40:00,left,30,3.9,102
2020-09-22 10:40:00,left,60,NA,102
2020-09-22 10:40:00,left,90,6.7,102
2020-09-22 11:00:00,right,30,4.6,102
2020-09-22 11:00:00,right,60,NA,102
2020-09-22 11:00:00,right,90,3.3,102
